# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/smartrics/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/smartrics/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [ ]:
import os
import subprocess

repo_dir = "/content/mistral-finetune"
repo_url = "https://github.com/smartrics/mistral-finetune.git"

if os.path.isdir(repo_dir):
    print("Directory 'mistral-finetune' exists. Pulling latest changes...")
    subprocess.run(["git", "-C", repo_dir, "pull"], check=True)
else:
    print("Directory 'mistral-finetune' does not exist. Cloning repository...")
    subprocess.run(["git", "clone", repo_url, repo_dir], check=True)
print("finished!")

Install all required dependencies:

In [ ]:
!pip install -r /content/mistral-finetune/requirements.txt

## Model download

In [ ]:
!pip install huggingface_hub

In [4]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

# Define the target path directly in /content/mistral_models
mistral_models_path = Path("/content/mistral_models/7B-Instruct-v0.3")
mistral_models_path.mkdir(parents=True, exist_ok=True)

# Download directly to the target directory
snapshot_download(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"],
    local_dir=mistral_models_path
)

print(f"Model downloaded to {mistral_models_path}")


## Dataset

Use the data in `/content/mistral-finetune/data`

In [6]:
!ls /content/mistral-finetune/data

prepare.py  test_data.jsonl  training_data.jsonl  validation_data.jsonl


In [7]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [ ]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


## Start training

In [9]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [10]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
data:
  instruct_data: "data/test_data.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "data/validation_data.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models/7B-Instruct-v0.3"  # Change to downloaded path
lora:
  rank: 64

# optim
seq_len: 8192
batch_size: 1
max_steps: 300
optim:
  lr: 6.e-5
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/mistral_models/7B-Instruct-v0.3_trained"  # Fill

wandb:
  project: None # your wandb project name
  run_name: "" # your wandb run name
  key: "" # your wandb api key
  offline: True

"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [11]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

import os
os.environ["WANDB_MODE"] = "disabled"


In [12]:
import os
import zipfile

# Define paths
zip_path = "/content/mistral_models/7B-Instruct-v0.3_trained.zip"
extract_path = "/content/mistral_models"

if os.path.exists(zip_path):
    print("Zip file found. Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
else:
    # start training
    print("Zip file not found. Removing existing model directory and starting training...")
    !rm -rf /content/mistral_models/7B-Instruct-v0.3_trained
    !torchrun -m train example.yaml
print("finished!")

Zip file not found. Removing existing model directory and starting training...
2025-03-10 19:58:35.091774: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 19:58:35.109139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741636715.131201    2179 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741636715.137877    2179 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 19:58:35.160275: I tensorflow/core/platform/cpu_featu

In [15]:
# Merge trained weights with the Base Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from safetensors.torch import load_file

# Define paths
BASE_MODEL_PATH = "/content/mistral_models/7B-Instruct-v0.3"
LORA_ADAPTER_PATH = "/content/mistral_models/7B-Instruct-v0.3_trained/checkpoints/checkpoint_000300/consolidated/lora.safetensors"
OUTPUT_MODEL_PATH = "/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", torch_dtype=torch.float16)

# Load LoRA weights from safetensors
lora_weights = load_file(LORA_ADAPTER_PATH)

# Manually update base model's state dict with LoRA weights
base_model.load_state_dict(lora_weights, strict=False)

# Save the fully merged model
base_model.save_pretrained(OUTPUT_MODEL_PATH)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)

print(f"✅ Model successfully merged and saved to: {OUTPUT_MODEL_PATH}")



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

✅ Model successfully merged and saved to: /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind


## Convert to Ollama
Ollama requires models in GGUF format

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Define paths
MODEL_PATH = "/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind"
GGUF_OUTPUT_PATH = "/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf"

# Load the model
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Export to GGUF
model.save_pretrained(GGUF_OUTPUT_PATH)
tokenizer.save_pretrained(GGUF_OUTPUT_PATH)

print(f"✅ Model converted to GGUF format: {GGUF_OUTPUT_PATH}")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model converted to GGUF format: /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf


In [ ]:
!tar cfvz /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind_gguf.tar.gz /content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind.gguf
print("finished")

In [25]:
import shutil

from google.colab import drive

drive.mount('/content/drive')


# Move the uploaded file
shutil.move('/content/mistral_models/Mistral-7B-Instruct-v0.3-tamarind_gguf.tar.gz', '/content/drive/My Drive/Mistral-7B-Instruct-v0.3-tamarind_gguf.tar.gz')



Mounted at /content/drive


'/content/drive/My Drive/Mistral-7B-Instruct-v0.3-tamarind_gguf.tar.gz'